Jay Alammar
1. [A Visual Guide to Using BERT for the First Time](https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)
2. [Visualizing A Neural Machine Translation Model (Mechanics of Seq2seq Models With Attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
3. [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

МФТИ
1. [Прикладное машинное обучение 5. Context based models. BERT overview](https://www.youtube.com/watch?v=1DygevyV2eA)

Ruben Winastwan
1. [Text Classification with BERT in PyTorch](https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f)

Example notebook
1. https://github.com/marcellusruben/medium-resources/blob/main/Text_Classification_BERT/bert_medium.ipynb





# create dataset

In [ ]:
!pip install vk_requests

In [ ]:
# import vk_requests
import time
import pandas as pd
from datetime import datetime
import os
import urllib.request

In [ ]:
token = '57654f39ff46921683dff5189464eaca5fe78b8780a8379102d4c0de2f03bb2be46e6972761f9217fa3ae'
api = vk_requests.create_api(service_token=token)

In [ ]:
# отсеиваем устаревшие группы
list_groups = [{'name': 'normilinet'}, 
               {'name': 'overhear'},
               {'name': 'inscrt'},
               ]
list_groups

[{'name': 'normilinet'}, {'name': 'overhear'}, {'name': 'inscrt'}]

In [ ]:
# берём посты групп за 2022 год
time_2022 = 1640950000
for group in list_groups:
  # постоянный сдвиг
  offset = 0
  # это время поста, оно должно быть больше чем 01.01.2022
  time_post = 2000000000
  print(group['name'])
  group['posts'] = []
  for _ in range(5):
    posts = api.wall.get(domain=group['name'], offset=offset, count=100)['items']
    
    posts_list = [post['text'] for post in posts]
    # в словаре добавляем посты к конкретной группе (к которой они принадлежат)
    group['posts'] += posts_list
    # сдвигаем
    offset += 100
    # смотрим какая дата у посленего взятого нами поста
    # time_post = posts[-1]['date']
    # try:
    #   time_post = posts[-1]['date']
    # except:
    #   break
    # спим
    time.sleep(0.33)

normilinet
overhear
inscrt


In [ ]:
full_list = []
for group in list_groups[:]:
  for post in group['posts']:
    # print(post)
    # index_sub_str = post.find('\n\n') # после \n\n идёт фио препода, поэтому отбросим эту часть
    full_list.append(post.replace('\n', ' '))

In [ ]:
len(list_groups[1]['posts'])

500

In [ ]:
len(full_list)

1500

In [ ]:
df_no_education = pd.DataFrame({'phrase': full_list})
df_education = pd.read_csv('phrase_of_teachers.csv').drop(['Unnamed: 0'], axis=1)

df_no_education['label'] = 0
df_education['label'] = 1

df_no_education = df_no_education.dropna().reset_index(drop=True)
df_education = df_education.dropna().reset_index(drop=True)

In [ ]:
df = pd.concat([df_no_education, df_education], ignore_index=True)
df

,phrase,label
0,Всем привет)Мужики какие у вас секреты не быть...,0
1,ГОЛАЯ ПРАВДА О ЖЕНСКОЙ УДАЛЕНКЕ (9 фактов от И...,0
2,"Задумалась о том, что пора начать экономить. ...",0
3,Когда в очередной раз ваша любимая спросит: «Я...,0
4,Всем привет Может кто-то сталкивался с такой п...,0
...,...,...
3232,✅ О нашем опыте участия в конкурсе рассказывае...,1
3233,В гимназию пришёл тираж первого номера первого...,1
3234,Эта неделя в гимназии - неделя кафедры физичес...,1
3235,Любая проверка знаний— это стресс для ребёнка....,1


In [ ]:
df.to_csv('pharse_dataset.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('pharse_dataset.csv').drop(['Unnamed: 0'], axis=1)
df = df.dropna().reset_index(drop=True)
df

,phrase,label
0,Всем привет)Мужики какие у вас секреты не быть...,0
1,ГОЛАЯ ПРАВДА О ЖЕНСКОЙ УДАЛЕНКЕ (9 фактов от И...,0
2,"Задумалась о том, что пора начать экономить. ...",0
3,Когда в очередной раз ваша любимая спросит: «Я...,0
4,Всем привет Может кто-то сталкивался с такой п...,0
...,...,...
3226,✅ О нашем опыте участия в конкурсе рассказывае...,1
3227,В гимназию пришёл тираж первого номера первого...,1
3228,Эта неделя в гимназии - неделя кафедры физичес...,1
3229,Любая проверка знаний— это стресс для ребёнка....,1


In [ ]:
df.isna().sum()

phrase    0
label     0
dtype: int64

# clear dataset

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts-ru

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 11.3 MB/s 


In [ ]:
import nltk
nltk.download('stopwords')
import pymorphy2
from nltk.corpus import stopwords
import re
from nltk.probability import FreqDist
import difflib
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
morph = pymorphy2.MorphAnalyzer()
stopwords_ru = stopwords.words("russian") + ['это', 'нею', 'самый', 'ещё', "нужно", 
                                             "очень", "свой", "ваш", "наш", "весь", 
                                             "всё", "который", "потому", "что", "поэтому",
                                             "почему", "просто", "вообще", 'жизнь', 
                                             'хороший', 'большой', 'сегодня', 'хотеть',
                                             'сказать', 'видеть', 'время', ]
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

In [ ]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if token not in stopwords_ru:
              tokens.append(token)
    return tokens
    # if len(tokens) > 2:
    #     return tokens
    # return None

In [ ]:
' '.join(lemmatize(df['phrase'].item()))

'привет мужик секрет скорострелый любовь каждый секрет полова мочь поделиться скорострелый подслушать'

In [ ]:
df['phrase'] = df['phrase'].apply(lambda x: ' '.join(lemmatize(x)))
df

,phrase,label
0,привет мужик секрет скорострелый любовь каждый...,0
1,голый правда женский удалёнка факт ирина год д...,0
2,задуматься пора начать экономить начать мочь э...,0
3,очередной любимый спросить «я красивый » отвеч...,0
4,привет мочь сталкиваться проблема отрастить не...,0
...,...,...
3226,✅ опыт участие конкурс рассказывать олеся алек...,1
3227,гимназия прийти тираж первый номер первый студ...,1
3228,неделя гимназия неделя кафедра физический куль...,1
3229,любой проверка знание стресс ребёнок особенно ...,1


In [ ]:
df.to_csv('clear_phrase_dataset.csv')

# learning

In [ ]:
# df = pd.read_csv('clear_phrase_dataset.csv').drop('Unnamed: 0', axis=1)
df = pd.read_csv('pharse_dataset.csv').drop('Unnamed: 0', axis=1)
df

,phrase,label
0,Всем привет)Мужики какие у вас секреты не быть...,0
1,ГОЛАЯ ПРАВДА О ЖЕНСКОЙ УДАЛЕНКЕ (9 фактов от И...,0
2,"Задумалась о том, что пора начать экономить. ...",0
3,Когда в очередной раз ваша любимая спросит: «Я...,0
4,Всем привет Может кто-то сталкивался с такой п...,0
...,...,...
3232,✅ О нашем опыте участия в конкурсе рассказывае...,1
3233,В гимназию пришёл тираж первого номера первого...,1
3234,Эта неделя в гимназии - неделя кафедры физичес...,1
3235,Любая проверка знаний— это стресс для ребёнка....,1


In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.isna().sum()

phrase    0
label     0
dtype: int64

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = list(df['label'])
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['phrase']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))
EPOCHS = 5
model = BertClassifier()
LR = 1e-6


2584 323 324


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train(model, 
      df_train, 
      df_val, 
      LR, 
      EPOCHS)
evaluate(model, df_test)

100%|██████████| 1292/1292 [09:22<00:00,  2.30it/s]


Epochs: 1 | Train Loss:  0.358 | Train Accuracy:  0.792 | Val Loss:  0.177 | Val Accuracy:  0.910


100%|██████████| 1292/1292 [09:21<00:00,  2.30it/s]


Epochs: 2 | Train Loss:  0.136 | Train Accuracy:  0.917 | Val Loss:  0.109 | Val Accuracy:  0.935


100%|██████████| 1292/1292 [09:21<00:00,  2.30it/s]


Epochs: 3 | Train Loss:  0.084 | Train Accuracy:  0.952 | Val Loss:  0.105 | Val Accuracy:  0.929


100%|██████████| 1292/1292 [09:21<00:00,  2.30it/s]


Epochs: 4 | Train Loss:  0.062 | Train Accuracy:  0.963 | Val Loss:  0.074 | Val Accuracy:  0.947


100%|██████████| 1292/1292 [09:21<00:00,  2.30it/s]


Epochs: 5 | Train Loss:  0.041 | Train Accuracy:  0.977 | Val Loss:  0.078 | Val Accuracy:  0.944
Test Accuracy:  0.917


In [ ]:
def predict_bert(text):
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  test_text = tokenizer(text, padding='max_length', max_length = 50, truncation=True, return_tensors="pt")
  
  with torch.no_grad():

    mask = test_text['attention_mask'].to('cpu')
    input_id = test_text['input_ids'].squeeze(1).to('cpu')
    model.to('cpu')

    output = model(input_id, mask)
    label_of_edu = 'education' if output.argmax(dim=1).item() else 'not education'

    print(f'ВСЕ ВЫХОДЫ: {output[0]}')
    print(f'КЛАСС: {label_of_edu}')

In [ ]:
text = 'мясо'
lem = ' '.join(lemmatize(text))

In [ ]:
predict_bert(text)

ВСЕ ВЫХОДЫ: tensor([3.3895, 1.6214, 0.0000, 0.0000, 0.0000])
КЛАСС: not education


In [ ]:
torch.save(model, '/content/drive/MyDrive/model_bert_nlp.pt')

# model = torch.load('/content/model.pt', map_location='cpu')
# model.eval()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
